In [1]:
# 패키지 로드
import os
import numpy as np
import pandas as pd
# import pymysql
import requests
import math
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [2]:
global_solo = pd.read_csv('projectData/global_merge.csv', index_col=0)
opgg_solo = pd.read_csv('projectData/opgg_merge.csv', index_col=0)
opgg_solo = opgg_solo.drop(columns='key')

In [3]:
global_solo =global_solo.rename(columns = {'tier':'g_tier', 'version':'g_version', 'rank':'g_rank'})

## 버전 11.11

In [4]:
global_solo_v11 = global_solo[global_solo['g_version'] == 11.11]

In [5]:
opgg_solo_v11 = opgg_solo[opgg_solo['patchIndex'] == 1111]

In [6]:
a_v11 = pd.merge(opgg_solo_v11, global_solo_v11, how='left', left_on='championId', right_on='key')
a_v11 = a_v11.drop(columns=['idx','patchIndex','championId','key','createdAt'])

## 버전 11.12

In [7]:
global_solo_v12 = global_solo[global_solo['g_version'] == 11.12]

In [8]:
opgg_solo_v12 = opgg_solo[opgg_solo['patchIndex'] == 1112]

In [9]:
a_v12 = pd.merge(opgg_solo_v12, global_solo_v12, how='left', left_on='championId', right_on='key')
a_v12 = a_v12.drop(columns=['idx','patchIndex','championId','key','createdAt'])

# 버전 11.13

In [10]:
global_solo_v13 = global_solo[global_solo['g_version'] == 11.13]

opgg_solo_v13 = opgg_solo[opgg_solo['patchIndex'] == 1113]

a_v13 = pd.merge(opgg_solo_v13, global_solo_v13, how='left', left_on='championId', right_on='key')
a_v13 = a_v13.drop(columns=['idx','patchIndex','championId','key','createdAt'])

In [11]:
# 버전 11.14

global_solo_v14 = global_solo[global_solo['g_version'] == 11.14]

opgg_solo_v14 = opgg_solo[opgg_solo['patchIndex'] == 1114]

a_v14 = pd.merge(opgg_solo_v14, global_solo_v14, how='left', left_on='championId', right_on='key')
a_v14 = a_v14.drop(columns=['idx','patchIndex','championId','key','createdAt'])

In [12]:
# 버전 11.15

global_solo_v15 = global_solo[global_solo['g_version'] == 11.15]

opgg_solo_v15 = opgg_solo[opgg_solo['patchIndex'] == 1115]

a_v15 = pd.merge(opgg_solo_v15, global_solo_v15, how='left', left_on='championId', right_on='key')
a_v15 = a_v15.drop(columns=['idx','patchIndex','championId','key','createdAt'])

In [13]:
# 버전 11.16

global_solo_v16 = global_solo[global_solo['g_version'] == 11.16]

opgg_solo_v16 = opgg_solo[opgg_solo['patchIndex'] == 1116]

a_v16 = pd.merge(opgg_solo_v16, global_solo_v16, how='left', left_on='championId', right_on='key')
a_v16 = a_v16.drop(columns=['idx','patchIndex','championId','key','createdAt'])

In [14]:
# 버전별 데이터 합쳐서 만들어주기
a_all = pd.concat([a_v11,a_v12,a_v13,a_v14,a_v15,a_v16])

In [15]:
# 확인해보기
a_all[a_all['name']=='제드']

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,name,g_version,g_rank,champion,g_tier,win_rate,pick,ban,pbi,games
0,MID,1,0.352734,1,0,1,2,제드,11.11,22,Zed,A,50.93,11.80,41.52,-1,"272,832"
218,MID,2,0.451905,1,0,1,0,제드,11.11,22,Zed,A,50.93,11.80,41.52,-1,"272,832"
434,MID,3,0.395300,1,0,2,-1,제드,11.11,22,Zed,A,50.93,11.80,41.52,-1,"272,832"
650,MID,4,0.402835,1,0,2,0,제드,11.11,22,Zed,A,50.93,11.80,41.52,-1,"272,832"
870,MID,5,0.346766,1,0,3,-1,제드,11.11,22,Zed,A,50.93,11.80,41.52,-1,"272,832"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2565,MID,10,0.320684,1,0,2,0,제드,11.16,13,Zed,A+,51.03,11.41,41.99,3,"249,629"
2566,MID,11,0.307987,1,0,2,0,제드,11.16,13,Zed,A+,51.03,11.41,41.99,3,"249,629"
2567,MID,12,0.313539,1,0,2,0,제드,11.16,13,Zed,A+,51.03,11.41,41.99,3,"249,629"
2568,MID,13,0.314917,1,0,2,0,제드,11.16,13,Zed,A+,51.03,11.41,41.99,3,"249,629"


In [16]:
a_all.to_csv('MLdata_champion.csv')

## 데이터 전처리

In [17]:
# 머신러닝 활용을 위해 포지션들 숫자로 바꿔주기
a_all['position'] = a_all['position'].str.replace('TOP','1')
a_all['position'] = a_all['position'].str.replace('MID','3')
a_all['position'] = a_all['position'].str.replace('JUNGLE','2')
a_all['position'] = a_all['position'].str.replace('ADC','4')
a_all['position'] = a_all['position'].str.replace('SUPPORT','5')
a_all['position'] =pd.to_numeric(a_all['position'])

In [22]:
a_all.to_csv('projectData/ML_DATA.csv')

In [24]:
a_all[a_all['g_version'] == 11.11]

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,name,g_version,g_rank,champion,g_tier,win_rate,pick,ban,pbi,games
0,3,1,0.352734,1,0,1,2,제드,11.11,22,Zed,A,50.93,11.80,41.52,-1,"272,832"
1,3,1,0.267511,1,0,2,-1,탈론,11.11,10,Talon,S-,52.66,5.23,6.22,9,"120,956"
2,3,1,0.256379,1,0,3,1,키아나,11.11,20,Qiyana,A,50.32,5.45,6.76,-4,"125,977"
3,3,1,0.238508,1,0,4,1,사일러스,11.11,8,Sylas,S-,50.60,16.24,21.44,-8,"375,660"
4,3,1,0.204850,1,0,5,-3,카타리나,11.11,5,Katarina,S,52.11,9.52,17.71,13,"220,124"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827,4,13,-0.259118,5,0,18,1,루시안,11.11,32,Lucian,C,48.15,5.14,3.18,-15,"119,848"
2828,4,13,-0.268859,5,0,19,2,야스오,11.11,16,Yasuo,A,51.33,9.58,17.49,4,"221,609"
2829,4,13,-0.273121,5,0,20,0,드레이븐,11.11,15,Draven,B+,50.69,4.37,2.93,-1,"101,901"
2830,4,13,-0.429642,5,0,21,1,아펠리오스,11.11,33,Aphelios,C,46.52,2.76,0.40,-12,"64,333"


In [25]:
a_all['lck_pick'] = 0

In [149]:
a_all2 = a_all.reset_index(drop=True)

In [27]:
LCK = pd.read_csv('projectData/LCK_RESULT.csv', index_col=0)
LCK['version'] = LCK['version'].str.replace('버전','')
LCK['version'] = LCK['version'].str.replace(' ','')

In [284]:
a_all2['lck_pick'] = 0

In [285]:
# 버전별로 데이터 구분해서 나눠주기
a_all2_v11 = a_all2[a_all2['g_version'] == 11.11]
a_all2_v12 = a_all2[a_all2['g_version'] == 11.12]
a_all2_v13 = a_all2[a_all2['g_version'] == 11.13]
a_all2_v14 = a_all2[a_all2['g_version'] == 11.14]
a_all2_v15 = a_all2[a_all2['g_version'] == 11.15]
a_all2_v16 = a_all2[a_all2['g_version'] == 11.16]

In [393]:
a_all2_v19 = a_all2[a_all2['g_version'] == 11.19]

In [391]:
len(a_all2_v16)

3087

In [293]:
# 버전에서 대회에 사용된 챔피언 확인
a = LCK[LCK['version'] == '11.11']['blue_champion'].unique()
b = LCK[LCK['version'] == '11.11']['red_champion'].unique()
세트, 오리아나, 

In [312]:
# 대회에서 사용된 챔피언들에 맞춰서 솔랭데이터에 라벨표시
for i in b:
    for n in range (0, len(a_all2_v16)):
        n 1, 2800
        iloc[(1,7)]
        if a_all2_v16.iloc[(n,7)] == f"{i}":
            a_all2_v16.iloc[(n,17)] = 1 

In [313]:
a_all2_vall = pd.concat([a_all2_v11,a_all2_v12,a_all2_v13,a_all2_v14,a_all2_v15,a_all2_v16])

In [320]:
a_all2_vall[(a_all2_vall['g_version'] == 11.16) & (a_all2_vall['lck_pick'] == 1)]

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,name,g_version,g_rank,champion,g_tier,win_rate,pick,ban,pbi,games,lck_pick
16431,3,1,-0.037097,4,-2,31,-19,갈리오,11.16,19,Galio,A,51.82,2.17,0.66,2,"47,436",1
16432,3,2,-0.073331,3,1,21,10,갈리오,11.16,19,Galio,A,51.82,2.17,0.66,2,"47,436",1
16433,3,3,0.020576,3,0,21,0,갈리오,11.16,19,Galio,A,51.82,2.17,0.66,2,"47,436",1
16434,3,4,0.077413,2,1,16,5,갈리오,11.16,19,Galio,A,51.82,2.17,0.66,2,"47,436",1
16435,3,5,0.022848,3,-1,19,-3,갈리오,11.16,19,Galio,A,51.82,2.17,0.66,2,"47,436",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19485,2,10,-0.434613,5,0,39,2,그웬,11.16,69,Gwen,C+,47.11,3.22,8.81,-13,"69,589",1
19486,2,11,-0.435327,5,0,38,1,그웬,11.16,69,Gwen,C+,47.11,3.22,8.81,-13,"69,589",1
19487,2,12,-0.436497,5,0,39,-1,그웬,11.16,69,Gwen,C+,47.11,3.22,8.81,-13,"69,589",1
19488,2,13,-0.426388,5,0,39,0,그웬,11.16,69,Gwen,C+,47.11,3.22,8.81,-13,"69,589",1


In [324]:
a_all2_vall = a_all2_vall.drop(columns='champion')

In [329]:
a_all2_vall.to_csv('MLdata_final.csv')

In [ ]:
constant_patch = requests.get('https://ddragon.leagueoflegends.com/api/versions.json').json()[0]

# 라이엇 개발자 페이지에서 최신 버전 한국 서버 챔피언 정보 가져오기
champion_info = requests.get(f'http://ddragon.leagueoflegends.com/cdn/{constant_patch}/data/ko_KR/champion.json').json()
champion_df = pd.DataFrame(champion_info['data']).T[['key','name']]
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [347]:
a_all2_vall2 = pd.merge(a_all2_vall,champion_df,  how='left', left_on='name', right_on='name')
a_all2_vall2 = a_all2_vall2.drop(columns='name')
a_all2_vall2

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,g_version,g_rank,g_tier,win_rate,pick,ban,pbi,games,lck_pick,key
0,3,1,0.352734,1,0,1,2,11.11,22,A,50.93,11.80,41.52,-1,"272,832",0,238
1,3,1,0.267511,1,0,2,-1,11.11,10,S-,52.66,5.23,6.22,9,"120,956",0,91
2,3,1,0.256379,1,0,3,1,11.11,20,A,50.32,5.45,6.76,-4,"125,977",0,246
3,3,1,0.238508,1,0,4,1,11.11,8,S-,50.60,16.24,21.44,-8,"375,660",1,517
4,3,1,0.204850,1,0,5,-3,11.11,5,S,52.11,9.52,17.71,13,"220,124",0,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19485,2,10,-0.434613,5,0,39,2,11.16,69,C+,47.11,3.22,8.81,-13,"69,589",1,887
19486,2,11,-0.435327,5,0,38,1,11.16,69,C+,47.11,3.22,8.81,-13,"69,589",1,887
19487,2,12,-0.436497,5,0,39,-1,11.16,69,C+,47.11,3.22,8.81,-13,"69,589",1,887
19488,2,13,-0.426388,5,0,39,0,11.16,69,C+,47.11,3.22,8.81,-13,"69,589",1,887


In [348]:
a_all2_vall2

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,g_version,g_rank,g_tier,win_rate,pick,ban,pbi,games,lck_pick,key
0,3,1,0.352734,1,0,1,2,11.11,22,A,50.93,11.80,41.52,-1,"272,832",0,238
1,3,1,0.267511,1,0,2,-1,11.11,10,S-,52.66,5.23,6.22,9,"120,956",0,91
2,3,1,0.256379,1,0,3,1,11.11,20,A,50.32,5.45,6.76,-4,"125,977",0,246
3,3,1,0.238508,1,0,4,1,11.11,8,S-,50.60,16.24,21.44,-8,"375,660",1,517
4,3,1,0.204850,1,0,5,-3,11.11,5,S,52.11,9.52,17.71,13,"220,124",0,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19485,2,10,-0.434613,5,0,39,2,11.16,69,C+,47.11,3.22,8.81,-13,"69,589",1,887
19486,2,11,-0.435327,5,0,38,1,11.16,69,C+,47.11,3.22,8.81,-13,"69,589",1,887
19487,2,12,-0.436497,5,0,39,-1,11.16,69,C+,47.11,3.22,8.81,-13,"69,589",1,887
19488,2,13,-0.426388,5,0,39,0,11.16,69,C+,47.11,3.22,8.81,-13,"69,589",1,887


In [349]:
a_all2_vall2['g_tier'] = a_all2_vall2['g_tier'].str.replace('S','1')
a_all2_vall2['g_tier'] = a_all2_vall2['g_tier'].str.replace('-','')
a_all2_vall2['g_tier'] = a_all2_vall2['g_tier'].str.replace('+','')


a_all2_vall2['g_tier'] = a_all2_vall2['g_tier'].str.replace('A','2')
a_all2_vall2['g_tier'] = a_all2_vall2['g_tier'].str.replace('B','3')
a_all2_vall2['g_tier'] = a_all2_vall2['g_tier'].str.replace('C','4')
a_all2_vall2['g_tier'] = a_all2_vall2['g_tier'].str.replace('D','5')


<ipython-input-349-2fdb2ceafe2a>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  a_all2_vall2['g_tier'] = a_all2_vall2['g_tier'].str.replace('+','')


In [350]:
a_all2_vall2

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,g_version,g_rank,g_tier,win_rate,pick,ban,pbi,games,lck_pick,key
0,3,1,0.352734,1,0,1,2,11.11,22,2,50.93,11.80,41.52,-1,"272,832",0,238
1,3,1,0.267511,1,0,2,-1,11.11,10,1,52.66,5.23,6.22,9,"120,956",0,91
2,3,1,0.256379,1,0,3,1,11.11,20,2,50.32,5.45,6.76,-4,"125,977",0,246
3,3,1,0.238508,1,0,4,1,11.11,8,1,50.60,16.24,21.44,-8,"375,660",1,517
4,3,1,0.204850,1,0,5,-3,11.11,5,1,52.11,9.52,17.71,13,"220,124",0,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19485,2,10,-0.434613,5,0,39,2,11.16,69,4,47.11,3.22,8.81,-13,"69,589",1,887
19486,2,11,-0.435327,5,0,38,1,11.16,69,4,47.11,3.22,8.81,-13,"69,589",1,887
19487,2,12,-0.436497,5,0,39,-1,11.16,69,4,47.11,3.22,8.81,-13,"69,589",1,887
19488,2,13,-0.426388,5,0,39,0,11.16,69,4,47.11,3.22,8.81,-13,"69,589",1,887


In [359]:
a_all2_vall2['games'] = a_all2_vall2['games'].str.replace(',','')
a_all2_vall2['games'] = pd.to_numeric(a_all2_vall2['games'])

In [360]:
a_all2_vall2

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,g_version,g_rank,g_tier,win_rate,pick,ban,pbi,games,lck_pick,key
0,3,1,0.352734,1,0,1,2,11.11,22,2,50.93,11.80,41.52,-1,272832,0,238
1,3,1,0.267511,1,0,2,-1,11.11,10,1,52.66,5.23,6.22,9,120956,0,91
2,3,1,0.256379,1,0,3,1,11.11,20,2,50.32,5.45,6.76,-4,125977,0,246
3,3,1,0.238508,1,0,4,1,11.11,8,1,50.60,16.24,21.44,-8,375660,1,517
4,3,1,0.204850,1,0,5,-3,11.11,5,1,52.11,9.52,17.71,13,220124,0,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19485,2,10,-0.434613,5,0,39,2,11.16,69,4,47.11,3.22,8.81,-13,69589,1,887
19486,2,11,-0.435327,5,0,38,1,11.16,69,4,47.11,3.22,8.81,-13,69589,1,887
19487,2,12,-0.436497,5,0,39,-1,11.16,69,4,47.11,3.22,8.81,-13,69589,1,887
19488,2,13,-0.426388,5,0,39,0,11.16,69,4,47.11,3.22,8.81,-13,69589,1,887


# 머신러닝 적용해보기

In [364]:
# 경고 메시지가 안나오게..

import warnings
warnings.filterwarnings('ignore')


# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# KFold
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


# 교차검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 저장
import pickle

# 그래프 설정
plt.rcParams['font.family']= 'Malgun Gothic'
# plt.rcParams['font.family']= 'AppleGothic'
plt.rcParams['font.size']  = 16
plt.rcParams['figure.figsize'] = (20,10)
plt.rcParams['axes.unicode_minus'] = False

In [476]:
x = a_all2_vall2.drop(['lck_pick'], axis=1)
y = a_all2_vall2['lck_pick']

display(x)
display(y)


,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,g_version,g_rank,g_tier,win_rate,pick,ban,pbi,games,key
0,3,1,0.352734,1,0,1,2,11.11,22,2,50.93,11.80,41.52,-1,272832,238
1,3,1,0.267511,1,0,2,-1,11.11,10,1,52.66,5.23,6.22,9,120956,91
2,3,1,0.256379,1,0,3,1,11.11,20,2,50.32,5.45,6.76,-4,125977,246
3,3,1,0.238508,1,0,4,1,11.11,8,1,50.60,16.24,21.44,-8,375660,517
4,3,1,0.204850,1,0,5,-3,11.11,5,1,52.11,9.52,17.71,13,220124,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19485,2,10,-0.434613,5,0,39,2,11.16,69,4,47.11,3.22,8.81,-13,69589,887
19486,2,11,-0.435327,5,0,38,1,11.16,69,4,47.11,3.22,8.81,-13,69589,887
19487,2,12,-0.436497,5,0,39,-1,11.16,69,4,47.11,3.22,8.81,-13,69589,887
19488,2,13,-0.426388,5,0,39,0,11.16,69,4,47.11,3.22,8.81,-13,69589,887


0        0
1        0
2        0
3        1
4        0
        ..
19485    1
19486    1
19487    1
19488    1
19489    1
Name: lck_pick, Length: 19490, dtype: int64

In [483]:
scaler1 = StandardScaler()
scaler1.fit(x)
x = scaler1.transform(x)
x


array([[ 0.15349757, -1.51145124,  1.46391967, ..., -0.19441961,
         1.26116195,  0.43710898],
       [ 0.15349757, -1.51145124,  1.10736898, ...,  0.72935673,
        -0.15564379, -0.33486511],
       [ 0.15349757, -1.51145124,  1.06079617, ..., -0.47155251,
        -0.10880439,  0.47912118],
       ...,
       [-0.54778412,  0.77624004, -1.83803528, ..., -1.30295122,
        -0.63483116,  3.84534834],
       [-0.54778412,  0.98421198, -1.79573954, ..., -1.30295122,
        -0.63483116,  3.84534834],
       [-0.54778412,  1.19218391, -1.71933358, ..., -1.30295122,
        -0.63483116,  3.84534834]])

In [367]:
#KFold 생성 : 교차 검증을 위해 데이터 구성을 하는 작업
fold1 = StratifiedKFold(n_splits=10)

In [368]:
# 하이퍼파라미터 튜닝
params = {
    'n_neighbors' : [1,2,3,4,5,6,7,8,9,10]
}

model1 =KNeighborsClassifier()
grid1  =GridSearchCV( model1, param_grid=params, scoring='accuracy', cv = fold1)
grid1.fit(x,y)
print(f'최적의 하이퍼 파라미터 : {grid1.best_params_}')
print(f'최적의 모델 평균 성능 : {grid1.best_score_}')

최적의 하이퍼 파라미터 : {'n_neighbors': 1}
최적의 모델 평균 성능 : 0.8997434581836838


In [369]:
params = {
    'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
}

model3 = SVC()
grid3 = GridSearchCV(model3, param_grid=params, scoring='accuracy', cv=fold1)
grid3.fit(x, y)
print(f'최적의 하이퍼 파라미터 : {grid3.best_params_}')
print(f'최적의 모델 평균 성능 : {grid3.best_score_}')

최적의 하이퍼 파라미터 : {'C': 10000}
최적의 모델 평균 성능 : 0.9324268855823499


In [370]:
params = {
    # 질문 깊이
    'max_depth' : [None, 1,2,3,4,5,6,7,8,9,10]
}

model4 = DecisionTreeClassifier()
grid4 = GridSearchCV(model4, param_grid=params, scoring='accuracy', cv=fold1)
grid4.fit(x, y)
print(f'최적의 하이퍼 파라미터 : {grid4.best_params_}')
print(f'최적의 모델 평균 성능 : {grid4.best_score_}')

최적의 하이퍼 파라미터 : {'max_depth': None}
최적의 모델 평균 성능 : 0.9384299640841458


In [371]:
params = {
    # 트리의 개수
    'n_estimators':[10,50,100,150,200],
    # 질문 깊이
    'max_depth' : [None, 1,2,3,4,5,6,7,8,9,10]
}

model5 = RandomForestClassifier()
grid5 = GridSearchCV(model5, param_grid=params, scoring='accuracy', cv=fold1)
grid5.fit(x, y)
print(f'최적의 하이퍼 파라미터 : {grid5.best_params_}')
print(f'최적의 모델 평균 성능 : {grid5.best_score_}')

최적의 하이퍼 파라미터 : {'max_depth': None, 'n_estimators': 200}
최적의 모델 평균 성능 : 0.9643406875320679


In [372]:
params = {
    # 트리의 개수
    'n_estimators' : [10, 50, 100, 150, 200],
    # 질문 깊이
    'max_depth' : [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

model6 = GradientBoostingClassifier()
grid6 = GridSearchCV(model6, param_grid=params, scoring='accuracy', cv=fold1)
grid6.fit(x, y)
print(f'최적의 하이퍼 파라미터 : {grid6.best_params_}')
print(f'최적의 모델 평균 성능 : {grid6.best_score_}')


최적의 하이퍼 파라미터 : {'max_depth': 6, 'n_estimators': 200}
최적의 모델 평균 성능 : 0.9929194458696766


# test 데이터 만들기

In [454]:
global_solo_tt = pd.read_csv('projectData/global_1119.csv', index_col=0)
opgg_solo_t = pd.read_csv('projectData/opgg_1119.csv', index_col=0)
opgg_solo_t = opgg_solo_t.drop(columns='key')

In [455]:
global_solo_tt =global_solo_tt.rename(columns = {'tier':'g_tier', 'version':'g_version', 'rank':'g_rank'})

In [456]:
global_solo_tt

,g_version,g_rank,champion,g_tier,win_rate,pick,ban,pbi,games,key
0,11.19,1,Xin Zhao,S+,52.58,5.73,9.11,12,"108,844",5.0
1,11.19,1,Vayne,S+,53.07,16.12,13.53,45,"309,322",67.0
2,11.19,1,Bard,S+,52.24,4.07,0.58,6,"75,667",432.0
3,11.19,1,Sett,S+,51.80,10.93,12.66,14,"205,801",875.0
4,11.19,2,Fiddlesticks,S,52.75,4.44,4.96,10,"84,330",9.0
...,...,...,...,...,...,...,...,...,...,...
152,11.19,83,Neeko,D+,50.37,0.77,0.13,0,"14,748",518.0
153,11.19,84,Gwen,C-,47.25,2.74,3.57,-10,"51,501",887.0
154,11.19,85,Ryze,D+,45.00,3.26,0.55,-19,"62,229",13.0
155,11.19,89,Sion,D,49.02,3.46,0.25,-6,"65,050",14.0


In [457]:
opgg_solo_t

,idx,patchIndex,championId,position,version,tierPoint,tier,tierRankDiff,createdAt,rank,rankDiff,name
0,294772,1119,1,MID,1,-0.338424,5,-1,2021-09-23 13:00:07,43,-5,애니
1,294996,1119,1,MID,2,-0.437983,5,0,2021-09-24 13:00:07,51,-8,애니
2,295204,1119,1,MID,3,-0.242933,5,0,2021-09-25 13:00:08,43,8,애니
3,295423,1119,1,MID,4,-0.283163,5,0,2021-09-26 13:00:07,44,-1,애니
4,295640,1119,1,MID,5,-0.295655,5,0,2021-09-27 13:00:07,44,0,애니
...,...,...,...,...,...,...,...,...,...,...,...,...
2154,295906,1119,887,JUNGLE,6,-0.487273,5,0,2021-09-28 13:00:07,41,0,그웬
2155,296122,1119,887,JUNGLE,7,-0.514441,5,0,2021-09-29 13:00:07,41,0,그웬
2156,296337,1119,887,JUNGLE,8,-0.516489,5,0,2021-09-30 13:00:07,41,0,그웬
2157,296552,1119,887,JUNGLE,9,-0.537293,5,0,2021-10-01 13:00:07,41,0,그웬


In [458]:
a_v19= pd.merge(opgg_solo_t, global_solo_tt, how='left', left_on='championId', right_on='key')
a_v19 = a_v19.drop(columns=['idx','patchIndex','championId','key','createdAt'])

In [459]:
a_v19

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,name,g_version,g_rank,champion,g_tier,win_rate,pick,ban,pbi,games
0,MID,1,-0.338424,5,-1,43,-5,애니,11.19,32,Annie,B+,52.58,2.91,0.98,6,"55,590"
1,MID,2,-0.437983,5,0,51,-8,애니,11.19,32,Annie,B+,52.58,2.91,0.98,6,"55,590"
2,MID,3,-0.242933,5,0,43,8,애니,11.19,32,Annie,B+,52.58,2.91,0.98,6,"55,590"
3,MID,4,-0.283163,5,0,44,-1,애니,11.19,32,Annie,B+,52.58,2.91,0.98,6,"55,590"
4,MID,5,-0.295655,5,0,44,0,애니,11.19,32,Annie,B+,52.58,2.91,0.98,6,"55,590"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154,JUNGLE,6,-0.487273,5,0,41,0,그웬,11.19,84,Gwen,C-,47.25,2.74,3.57,-10,"51,501"
2155,JUNGLE,7,-0.514441,5,0,41,0,그웬,11.19,84,Gwen,C-,47.25,2.74,3.57,-10,"51,501"
2156,JUNGLE,8,-0.516489,5,0,41,0,그웬,11.19,84,Gwen,C-,47.25,2.74,3.57,-10,"51,501"
2157,JUNGLE,9,-0.537293,5,0,41,0,그웬,11.19,84,Gwen,C-,47.25,2.74,3.57,-10,"51,501"


# 전처리

In [460]:
# 머신러닝 활용을 위해 포지션들 숫자로 바꿔주기
a_v19['position'] = a_v19['position'].str.replace('TOP','1')
a_v19['position'] = a_v19['position'].str.replace('MID','3')
a_v19['position'] = a_v19['position'].str.replace('JUNGLE','2')
a_v19['position'] = a_v19['position'].str.replace('ADC','4')
a_v19['position'] = a_v19['position'].str.replace('SUPPORT','5')
a_v19['position'] =pd.to_numeric(a_v19['position'])

In [422]:
a_v19

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,name,g_version,g_rank,champion,g_tier,win_rate,pick,ban,pbi,games
0,3,1,-0.338424,5,-1,43,-5,애니,11.19,32,Annie,B+,52.58,2.91,0.98,6,"55,590"
1,3,2,-0.437983,5,0,51,-8,애니,11.19,32,Annie,B+,52.58,2.91,0.98,6,"55,590"
2,3,3,-0.242933,5,0,43,8,애니,11.19,32,Annie,B+,52.58,2.91,0.98,6,"55,590"
3,3,4,-0.283163,5,0,44,-1,애니,11.19,32,Annie,B+,52.58,2.91,0.98,6,"55,590"
4,3,5,-0.295655,5,0,44,0,애니,11.19,32,Annie,B+,52.58,2.91,0.98,6,"55,590"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154,2,6,-0.487273,5,0,41,0,그웬,11.19,84,Gwen,C-,47.25,2.74,3.57,-10,"51,501"
2155,2,7,-0.514441,5,0,41,0,그웬,11.19,84,Gwen,C-,47.25,2.74,3.57,-10,"51,501"
2156,2,8,-0.516489,5,0,41,0,그웬,11.19,84,Gwen,C-,47.25,2.74,3.57,-10,"51,501"
2157,2,9,-0.537293,5,0,41,0,그웬,11.19,84,Gwen,C-,47.25,2.74,3.57,-10,"51,501"


In [433]:
a_v19 = a_v19.drop(columns='name')

In [461]:
a_v19['g_tier'] = a_v19['g_tier'].str.replace('S','1')
a_v19['g_tier'] = a_v19['g_tier'].str.replace('-','')
a_v19['g_tier'] = a_v19['g_tier'].str.replace('+','')


a_v19['g_tier'] = a_v19['g_tier'].str.replace('A','2')
a_v19['g_tier'] = a_v19['g_tier'].str.replace('B','3')
a_v19['g_tier'] = a_v19['g_tier'].str.replace('C','4')
a_v19['g_tier'] = a_v19['g_tier'].str.replace('D','5')


In [462]:
a_v19['games'] = a_v19['games'].str.replace(',','')
a_v19['games'] = pd.to_numeric(a_v19['games'])

In [463]:
a_v19

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,name,g_version,g_rank,champion,g_tier,win_rate,pick,ban,pbi,games
0,3,1,-0.338424,5,-1,43,-5,애니,11.19,32,Annie,3,52.58,2.91,0.98,6,55590
1,3,2,-0.437983,5,0,51,-8,애니,11.19,32,Annie,3,52.58,2.91,0.98,6,55590
2,3,3,-0.242933,5,0,43,8,애니,11.19,32,Annie,3,52.58,2.91,0.98,6,55590
3,3,4,-0.283163,5,0,44,-1,애니,11.19,32,Annie,3,52.58,2.91,0.98,6,55590
4,3,5,-0.295655,5,0,44,0,애니,11.19,32,Annie,3,52.58,2.91,0.98,6,55590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154,2,6,-0.487273,5,0,41,0,그웬,11.19,84,Gwen,4,47.25,2.74,3.57,-10,51501
2155,2,7,-0.514441,5,0,41,0,그웬,11.19,84,Gwen,4,47.25,2.74,3.57,-10,51501
2156,2,8,-0.516489,5,0,41,0,그웬,11.19,84,Gwen,4,47.25,2.74,3.57,-10,51501
2157,2,9,-0.537293,5,0,41,0,그웬,11.19,84,Gwen,4,47.25,2.74,3.57,-10,51501


In [464]:
a_v19 = pd.merge(a_v19,champion_df,  how='left', left_on='name', right_on='name')

In [465]:
a_v19 = a_v19.drop(columns=['name','champion'])

In [470]:
a_v19_test = a_v19[a_v19['version']== 10]
a_v19_test

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,g_version,g_rank,g_tier,win_rate,pick,ban,pbi,games,key
9,3,10,-0.187872,4,0,35,1,11.19,32,3,52.58,2.91,0.98,6,55590,1
19,2,10,-0.144048,4,0,25,-1,11.19,23,3,51.36,1.67,1.13,1,31693,2
29,3,10,0.019760,3,0,22,-2,11.19,23,2,51.33,2.67,0.61,2,50946,3
39,5,10,-0.200155,4,0,29,1,11.19,23,2,51.33,2.67,0.61,2,50946,3
49,3,10,0.011843,3,0,23,1,11.19,59,3,49.72,4.11,0.26,-4,78465,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2118,5,10,-0.475145,5,0,42,0,11.19,1,1,51.80,10.93,12.66,14,205801,875
2128,1,10,0.066150,3,0,20,0,11.19,40,4,50.17,4.13,2.92,-2,78580,876
2138,2,10,-0.090733,3,1,23,0,11.19,40,4,50.17,4.13,2.92,-2,78580,876
2148,1,10,-0.197169,4,0,41,0,11.19,84,4,47.25,2.74,3.57,-10,51501,887


In [471]:
a_v19_test = a_v19_test.reset_index()

In [474]:
a_v19_test = a_v19_test.drop(columns='index')
a_v19_test

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,g_version,g_rank,g_tier,win_rate,pick,ban,pbi,games,key
0,3,10,-0.187872,4,0,35,1,11.19,32,3,52.58,2.91,0.98,6,55590,1
1,2,10,-0.144048,4,0,25,-1,11.19,23,3,51.36,1.67,1.13,1,31693,2
2,3,10,0.019760,3,0,22,-2,11.19,23,2,51.33,2.67,0.61,2,50946,3
3,5,10,-0.200155,4,0,29,1,11.19,23,2,51.33,2.67,0.61,2,50946,3
4,3,10,0.011843,3,0,23,1,11.19,59,3,49.72,4.11,0.26,-4,78465,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,5,10,-0.475145,5,0,42,0,11.19,1,1,51.80,10.93,12.66,14,205801,875
211,1,10,0.066150,3,0,20,0,11.19,40,4,50.17,4.13,2.92,-2,78580,876
212,2,10,-0.090733,3,1,23,0,11.19,40,4,50.17,4.13,2.92,-2,78580,876
213,1,10,-0.197169,4,0,41,0,11.19,84,4,47.25,2.74,3.57,-10,51501,887


In [477]:
test_x = a_v19_test

In [479]:
best3 = grid3.best_estimator_
best5 = grid5.best_estimator_
best6 = grid6.best_estimator_

In [480]:
print(best3)
print(best5)
print(best6)

SVC(C=10000)
RandomForestClassifier(n_estimators=200)
GradientBoostingClassifier(max_depth=6, n_estimators=200)


In [525]:
r3 = cross_val_score(best3, x, y, scoring='f1', cv=fold1 )
r5 = cross_val_score(best5, x, y, scoring='f1', cv=fold1 )
r6 = cross_val_score(best6, x, y, scoring='f1', cv=fold1 )

In [526]:
print(r3.mean())
print(r5.mean())
print(r6.mean())

0.9099814613985521
0.9504336698137896
0.9899045772538108


In [484]:
best6.fit(x,y)

GradientBoostingClassifier(max_depth=6, n_estimators=200)

In [487]:
best3.fit(x,y)

SVC(C=10000)

In [488]:
best5.fit(x,y)

RandomForestClassifier(n_estimators=200)

In [485]:
pred1 = best6.predict(x)
pred1

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [486]:
result1 = accuracy_score( y, pred1)
result1

1.0

In [ ]:
test_x

In [489]:
scaler1 = StandardScaler()
scaler1.fit(test_x)
test_x = scaler1.transform(test_x)
test_x


array([[ 0.13848936,  0.        , -0.5895182 , ...,  0.48227947,
        -0.62122739, -0.79489799],
       [-0.55395746,  0.        , -0.40017348, ...,  0.00750801,
        -0.93039566, -0.78979838],
       [ 0.13848936,  0.        ,  0.30757033, ...,  0.1024623 ,
        -0.6813093 , -0.78469878],
       ...,
       [-0.55395746,  0.        , -0.16982189, ..., -0.27735486,
        -0.32379346,  3.66725652],
       [-1.24640428,  0.        , -0.62968516, ..., -1.03698919,
        -0.67412897,  3.72335217],
       [-0.55395746,  0.        , -1.92192791, ..., -1.03698919,
        -0.67412897,  3.72335217]])

In [513]:
pred2 = best6.predict(test_x)
pred2

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1], dtype=int64)

In [515]:
pred3 = best5.predict(test_x)
pred3

array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [519]:
pred4 = best3.predict(test_x)
pred4

array([0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], dtype=int64)

In [492]:
a1 = best6.predict_proba(test_x)
proba_df = pd.DataFrame(a1, columns = ['0','1'])
proba_df['target'] = pred2
proba_df

,0,1,target
0,0.056599,0.943401,1
1,0.040272,0.959728,1
2,0.018733,0.981267,1
3,0.003791,0.996209,1
4,0.311838,0.688162,1
...,...,...,...
210,0.000163,0.999837,1
211,0.923088,0.076912,0
212,0.923474,0.076526,0
213,0.037591,0.962409,1


In [496]:
pred2

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1], dtype=int64)

In [499]:
a_v19_test

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,g_version,g_rank,g_tier,win_rate,pick,ban,pbi,games,key
0,3,10,-0.187872,4,0,35,1,11.19,32,3,52.58,2.91,0.98,6,55590,1
1,2,10,-0.144048,4,0,25,-1,11.19,23,3,51.36,1.67,1.13,1,31693,2
2,3,10,0.019760,3,0,22,-2,11.19,23,2,51.33,2.67,0.61,2,50946,3
3,5,10,-0.200155,4,0,29,1,11.19,23,2,51.33,2.67,0.61,2,50946,3
4,3,10,0.011843,3,0,23,1,11.19,59,3,49.72,4.11,0.26,-4,78465,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,5,10,-0.475145,5,0,42,0,11.19,1,1,51.80,10.93,12.66,14,205801,875
211,1,10,0.066150,3,0,20,0,11.19,40,4,50.17,4.13,2.92,-2,78580,876
212,2,10,-0.090733,3,1,23,0,11.19,40,4,50.17,4.13,2.92,-2,78580,876
213,1,10,-0.197169,4,0,41,0,11.19,84,4,47.25,2.74,3.57,-10,51501,887


In [503]:



.sort_values('key')

,key,name
Annie,1,애니
Olaf,2,올라프
Galio,3,갈리오
TwistedFate,4,트위스티드 페이트
XinZhao,5,신 짜오
Urgot,6,우르곳
Leblanc,7,르블랑
Vladimir,8,블라디미르
Fiddlesticks,9,피들스틱
Kayle,10,케일


In [502]:
pd.set_option('display.max_rows', None)

In [522]:
a_v19_test['result3'] = result3

In [520]:
result3 = []

In [521]:
for i in pred4:
    result3.append(i)

In [511]:
pd.merge(a_v19_test, champion_df, how='left', left_on='key', right_on='key')

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,g_version,g_rank,g_tier,win_rate,pick,ban,pbi,games,key,result,name
0,3,10,-0.187872,4,0,35,1,11.19,32,3,52.58,2.91,0.98,6,55590,1,1,애니
1,2,10,-0.144048,4,0,25,-1,11.19,23,3,51.36,1.67,1.13,1,31693,2,1,올라프
2,3,10,0.019760,3,0,22,-2,11.19,23,2,51.33,2.67,0.61,2,50946,3,1,갈리오
3,5,10,-0.200155,4,0,29,1,11.19,23,2,51.33,2.67,0.61,2,50946,3,1,갈리오
4,3,10,0.011843,3,0,23,1,11.19,59,3,49.72,4.11,0.26,-4,78465,4,1,트위스티드 페이트
5,2,10,0.182268,2,0,11,-1,11.19,1,1,52.58,5.73,9.11,12,108844,5,0,신 짜오
6,1,10,-0.119064,4,0,34,1,11.19,21,2,52.40,3.42,1.31,6,64375,6,0,우르곳
7,3,10,0.327165,1,0,3,0,11.19,30,2,50.10,8.46,17.09,-6,161542,7,0,르블랑
8,1,10,-0.372622,5,0,52,0,11.19,13,2,51.73,3.64,5.21,4,69482,8,0,블라디미르
9,3,10,-0.300859,5,0,45,0,11.19,13,2,51.73,3.64,5.21,4,69482,8,0,블라디미르


In [524]:
pd.merge(a_v19_test, champion_df, how='left', left_on='key', right_on='key')

,position,version,tierPoint,tier,tierRankDiff,rank,rankDiff,g_version,g_rank,g_tier,win_rate,pick,ban,pbi,games,key,result,result2,result3,name
0,3,10,-0.187872,4,0,35,1,11.19,32,3,52.58,2.91,0.98,6,55590,1,1,0,0,애니
1,2,10,-0.144048,4,0,25,-1,11.19,23,3,51.36,1.67,1.13,1,31693,2,1,0,0,올라프
2,3,10,0.019760,3,0,22,-2,11.19,23,2,51.33,2.67,0.61,2,50946,3,1,0,1,갈리오
3,5,10,-0.200155,4,0,29,1,11.19,23,2,51.33,2.67,0.61,2,50946,3,1,1,1,갈리오
4,3,10,0.011843,3,0,23,1,11.19,59,3,49.72,4.11,0.26,-4,78465,4,1,1,1,트위스티드 페이트
5,2,10,0.182268,2,0,11,-1,11.19,1,1,52.58,5.73,9.11,12,108844,5,0,0,0,신 짜오
6,1,10,-0.119064,4,0,34,1,11.19,21,2,52.40,3.42,1.31,6,64375,6,0,0,0,우르곳
7,3,10,0.327165,1,0,3,0,11.19,30,2,50.10,8.46,17.09,-6,161542,7,0,0,0,르블랑
8,1,10,-0.372622,5,0,52,0,11.19,13,2,51.73,3.64,5.21,4,69482,8,0,0,0,블라디미르
9,3,10,-0.300859,5,0,45,0,11.19,13,2,51.73,3.64,5.21,4,69482,8,0,0,0,블라디미르
